In [2]:
!pip install --upgrade torch torchvision pytorch-ignite transformers datasets
!pip install git+https://github.com/microsoft/microxcaling-emulation.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [pytorch-ignite] 3/5 [transformers]
  Cloning https://github.com/microsoft/microxcaling-emulation.git to /tmp/pip-req-build-_0y_m271
  Running command git clone --filter=blob:none --quiet https://github.com/microsoft/microxcaling-emulation.git /tmp/pip-req-build-_0y_m271
Username for 'https://github.com': ^C
ERROR: Operation cancelled by user


In [6]:
import torch
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

from ignite.engine import create_supervised_evaluator
from ignite.metrics import TopKCategoricalAccuracy

import os
import shutil
from pathlib import Path


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# mobilenet_v2 = models.mobilenet_v3_large(pretrained=True).to(device).eval()
resnet50     = models.resnet50(pretrained=True).to(device).eval()


Downloading: "https://s3.amazonaws.com/pytorch/models/resnet50-19c8e357.pth" to /home/ashvin/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 97.8M/97.8M [00:02<00:00, 34.4MB/s]


In [ ]:
!huggingface-cli login

In [1]:
from datasets import load_dataset

# Fetch ImageNet-100 (resized already)
ds = load_dataset("clane9/imagenet-100", split="validation")
print(ds)  # ~5000 images


Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

Dataset({
    features: ['image', 'label'],
    num_rows: 5000
})


In [2]:
from pathlib import Path
from PIL import Image

ds = ds.remove_columns(["pixel_values"]) if "pixel_values" in ds.column_names else ds
out_dir = Path("imagenet100_subset/val")
out_dir.mkdir(parents=True, exist_ok=True)

label_names = ds.features["label"].names

for i, ex in enumerate(ds):
    img: Image.Image = ex["image"]
    lbl = label_names[ex["label"]]
    folder = out_dir / lbl
    folder.mkdir(exist_ok=True, parents=True)
    img.save(folder / f"{i}.png")


In [3]:
from ignite.metrics.metric import Metric
import torch

class ConfidenceMetric(Metric):
    def reset(self):
        self._conf_sum = 0.0
        self._count = 0

    def update(self, output):
        y_pred, _ = output
        probs = torch.softmax(y_pred, dim=1)
        max_conf = probs.max(dim=1)[0]
        self._conf_sum += max_conf.sum().item()
        self._count += max_conf.size(0)

    def compute(self):
        return self._conf_sum / self._count

class EntropyMetric(Metric):
    def reset(self):
        self._ent_sum = 0.0
        self._count = 0

    def update(self, output):
        y_pred, _ = output
        probs = torch.softmax(y_pred, dim=1)
        entropy = -(probs * probs.log()).sum(dim=1)
        self._ent_sum += entropy.sum().item()
        self._count += entropy.size(0)

    def compute(self):
        return self._ent_sum / self._count

In [4]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from ignite.engine import create_supervised_evaluator
from ignite.metrics import TopKCategoricalAccuracy, Loss
import torchvision.models as models
import torch
import torch.nn.functional as F

# Preprocessing
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

# Dataset & Loader
val_ds = datasets.ImageFolder(root="imagenet100_subset/val", transform=transform)
loader = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=4)

# Device & Model Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

mobilenet = models.mobilenet_v2(pretrained=True).to(device).eval()
resnet    = models.resnet50(pretrained=True).to(device).eval()

# Metrics
metrics = {
    "top1": TopKCategoricalAccuracy(k=1),
    "top5": TopKCategoricalAccuracy(k=5),
    "xent": Loss(F.cross_entropy),
    "confidence": ConfidenceMetric(),
    "entropy": EntropyMetric(),
}

# Evaluators
evaluator_mn = create_supervised_evaluator(mobilenet, metrics=metrics, device=device)
evaluator_rn = create_supervised_evaluator(resnet, metrics=metrics, device=device)

# Run
mn_state = evaluator_mn.run(loader)
rn_state = evaluator_rn.run(loader)

# Output
def print_metrics(name, state):
    print(f"{name} Top-1:           {state.metrics['top1']:.4f}")
    print(f"{name} Top-5:           {state.metrics['top5']:.4f}")
    print(f"{name} Cross-Entropy:   {state.metrics['xent']:.4f}")
    print(f"{name} Mean Confidence: {state.metrics['confidence']:.4f}")
    print(f"{name} Prediction Entropy: {state.metrics['entropy']:.4f}")

print_metrics("MobileNet-v2", mn_state)
print_metrics("ResNet50", rn_state)

/home/ashvin/.conda/envs/mx-scaling/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ashvin/.conda/envs/mx-scaling/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/ashvin/.conda/envs/mx-scaling/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. 

MobileNet-v2 Top-1:           0.0002
MobileNet-v2 Top-5:           0.0010
MobileNet-v2 Cross-Entropy:   17.0375
MobileNet-v2 Mean Confidence: 0.6758
MobileNet-v2 Prediction Entropy: 1.2750
ResNet50 Top-1:           0.0000
ResNet50 Top-5:           0.0010
ResNet50 Cross-Entropy:   16.0149
ResNet50 Mean Confidence: 0.7305
ResNet50 Prediction Entropy: 1.0694


In [5]:
from microxcaling.mx_mapping import inject_pytorch_ops
from microxcaling.formats import E4M3, BF8  # or E5M2, etc.

def inject_microxcaling(fmt_name):
    if fmt_name == "mx-e4m3":
        inject_pytorch_ops(E4M3)
    elif fmt_name == "mx-bf8":
        inject_pytorch_ops(BF8)
    else:
        pass  # Do nothing for torch native formats


ModuleNotFoundError: No module named 'microxcaling'

In [17]:
import microxcaling

ModuleNotFoundError: No module named 'microxcaling'